In [1]:
# **PySpark**: Pyspark Take Home Assignment

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg

spark = SparkSession.\
        builder.\
        appName("pyspark-take-home-assingment").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

23/03/06 20:17:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
## 3. The Data
# Read the csv data 

In [4]:
df_data = spark.read.option("header",True).csv("data/Pokemon.csv")

In [5]:
df_data.count()

800

In [6]:
len(df_data.columns)

13

In [7]:
df_data.printSchema()

root
 |-- #: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Type 1: string (nullable = true)
 |-- Type 2: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- HP: string (nullable = true)
 |-- Attack: string (nullable = true)
 |-- Defense: string (nullable = true)
 |-- Sp. Atk: string (nullable = true)
 |-- Sp. Def: string (nullable = true)
 |-- Speed: string (nullable = true)
 |-- Generation: string (nullable = true)
 |-- Legendary: string (nullable = true)



In [8]:
## 1. what are the top 5 strongest non-legendary monsters?

In [9]:
df_non_legendary_pokemon_df_data = df_data.filter(df_data["Legendary"] == False).orderBy(col("Total").desc())

In [10]:
top_strongest_non_legendary_pokemon = df_non_legendary_pokemon_df_data.select(col("name"), col("`Type 1`")).limit(5).show(5, False)

+-----------------------+------+
|name                   |Type 1|
+-----------------------+------+
|TyranitarMega Tyranitar|Rock  |
|MetagrossMega Metagross|Steel |
|GarchompMega Garchomp  |Dragon|
|SalamenceMega Salamence|Dragon|
|Slaking                |Normal|
+-----------------------+------+



In [11]:
# 2. Which Pokemon type has the highest average HP?

In [12]:
df_average_hp_per_type = df_data.filter(col("`Type 1`").isNotNull()).groupBy(col("`Type 1`")) \
                        .agg(avg(col("HP"))).withColumnRenamed("avg(HP)", "avg_hp").withColumnRenamed("`Type 1`", "type")

df_average_hp_per_type.orderBy(col("avg_hp").desc()).limit(1).show(1, False)

+------+-------+
|Type 1|avg_hp |
+------+-------+
|Dragon|83.3125|
+------+-------+



In [13]:
# 3. Which is the most common special Attack?

In [14]:

df_special_attack_occurances = df_data.filter(col("`Type 1`").isNotNull()).groupBy(col("`Type 1`")).count() \
                                .withColumnRenamed("count", "occurance").withColumnRenamed("`Type 1`", "type")

most_common_special_attack = df_special_attack_occurances.orderBy(col("occurance").desc()).limit(1).show(1, False)

+------+---------+
|Type 1|occurance|
+------+---------+
|Water |112      |
+------+---------+

